In [1]:
import pandas as pd
from epiweeks import Week
from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados

In [2]:
casos_confirmados = CasosConfirmados(force=True, hard=True)
casos_confirmados.shape()

current checksum: 7419d767b71babed09cee3d9525ac6f7efcb2da62488789444f86563af4cc68f
Parece que o arquivo C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\Casos confirmados.xlsx sofreu alterações, considere usar o método update ou o passe force=True
Utilizando o método update
Atualizando o arquivo {'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} com o C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\Casos confirmados.xlsx...
Casos excluidos: 0
Obitos excluidos: 0
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} salvo e C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\casos_confirmados_checksum atualizado
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19data

(955247, 22674)

In [3]:
casos = casos_confirmados.get_casos()

In [4]:
sivep = pd.read_excel('output/TABELA_testes_sivep.xlsx')
notifica = pd.read_excel('output/TABELA_testes_notifica.xlsx')
notifica = notifica.set_index('SE')
sivep = sivep.set_index('SE')

In [5]:
casos['dt_diag'] = casos['dt_diag'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
casos['is'] = casos['is'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [6]:
casos['se_diag'] = casos['dt_diag'].apply(lambda x: Week.fromdate(x))
casos.loc[~casos['is'].isnull(), 'se_is'] = casos['is'].loc[~casos['is'].isnull()].apply(lambda x: Week.fromdate(x))

In [7]:
tabela_diag = casos.groupby(['se_diag'])[['nome']].count()

In [8]:
tabela_is = casos.groupby(['se_is'])[['nome']].count()
for row in tabela_is.index.values:
    if row.year < 2020:
        tabela_is = tabela_is.drop(row)

In [9]:
tabela = pd.DataFrame(sivep['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SRAG'})
notifica = pd.DataFrame(notifica['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SG'})
tabela = tabela.merge(how = 'left', on = tabela.index, right = notifica)
tabela = tabela.rename(columns={'key_0': 'index'})

In [10]:
tabela_diag = tabela_diag.rename(columns={'nome': 'Diagnóstico'})
tabela_is = tabela_is.rename(columns={'nome':'Primeiros sintomas'})

In [11]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_diag.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_diag = tabela_diag.set_index(index_list)
tabela_diag = tabela_diag.reset_index()
tabela = tabela.merge(how='left', on='index', right = tabela_diag)

C:\Users\est.joaosilva\Anaconda3\lib\site-packages\numpy\core\numeric.py:2378: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [12]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_is.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_is = tabela_is.set_index(index_list)
tabela_is = tabela_is.reset_index()
tabela = tabela.merge(how = 'left', on = 'index', right = tabela_is)

In [13]:
notifica = notifica.reset_index().rename(columns={'Positividade SG':'TEMP'})
tabela = tabela.merge(how = 'left', on = tabela.index, right = notifica)
tabela = tabela[['Primeiros sintomas', 'Diagnóstico', 'Positividade SG', 'Positividade SRAG', 'SE']]
tabela = tabela.set_index('SE')

In [14]:
tabela = tabela.append(pd.DataFrame(tabela.sum(0)).transpose())

In [16]:
tabela

,Primeiros sintomas,Diagnóstico,Positividade SG,Positividade SRAG
202001,239.0,NaN,0.666667,0.111111
202002,147.0,NaN,0.500000,0.033333
202003,75.0,NaN,0.750000,0.037037
202004,52.0,NaN,0.600000,NaN
202005,100.0,NaN,0.571429,NaN
...,...,...,...,...
202115,15269.0,20335.0,0.397965,0.605877
202116,13436.0,17680.0,0.409730,0.531623
202117,5143.0,18003.0,0.413986,0.401589
202118,6.0,823.0,0.385373,0.083333


In [15]:
tabela.to_excel('output/TABELA_casos_apresentacao.xlsx', index = False)